In [1]:
import pandas as  pd
import os

In [ ]:
dir = "Extracted"
sheet = "Page 4 Generator Data"



for filename in os.listdir(dir):
    if filename.endswith('.xlsx'):
        filePath = os.path.join(dir,filename)
        excelFile = pd.read_excel(filePath,sheet_name=sheet,header=None)
       
        row_index = excelFile[excelFile.iloc[:,0]=='Plant Id'].index[0]
        excelFile.columns = excelFile.iloc[row_index]
        excelFile.columns = [col.replace('\n',' ') for col in excelFile.columns]
        excelFile = excelFile.iloc[row_index+1:,:]

        print(filename, row_index)
        excelFile.to_csv('csv/'+filename.removesuffix('.xlsx')+'.csv',index=None)